In [1]:
!pip install pandas
!pip install pyalgotrading


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
API_KEY = "3X9U3T4JEFONNL2N"

In [3]:
import pandas as pd
import requests

class ScriptData:
    def __init__(self):
        self.raw_scripts = {}
        self.scripts = {}

    def fetch_intraday_data(self, script):
        url = 'https://www.alphavantage.co/query'
        params = {
            "function": "TIME_SERIES_INTRADAY",
            "symbol": script,
            "apikey": API_KEY,
            "interval": "60min"
        }

        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
        except requests.exceptions.HTTPError as e:
            print("Cannot fetch data from Alpha Vantage")
            return

        data = response.json()
        data = data["Time Series (60min)"]
        self.raw_scripts[script] = {"timestamp": [], "open": [], "high": [], "low": [], "close": [], "volume": []}
        for timestamp, values in data.items():
            self.raw_scripts[script]["timestamp"].append(pd.Timestamp(timestamp))
            self.raw_scripts[script]["open"].append(float(values["1. open"]))
            self.raw_scripts[script]["high"].append(float(values["2. high"]))
            self.raw_scripts[script]["low"].append(float(values["3. low"]))
            self.raw_scripts[script]["close"].append(float(values["4. close"]))
            self.raw_scripts[script]["volume"].append(int(values["5. volume"]))

        for key, value in self.raw_scripts[script].items():
            self.raw_scripts[script][key] = reversed(value)

    def convert_intraday_data(self, script):
        if script not in self.raw_scripts.keys():
            print("Please fetch the data for this script first")
            return

        self.scripts[script] = pd.DataFrame.from_dict(self.raw_scripts[script]).sort_values(by=["timestamp"])

    def __getitem__(self, item):
        return self.scripts[item]

    def __setitem__(self, key, value):
        self.scripts[key] = value

    def __contains__(self, item):
        return item in self.scripts.keys()

In [4]:
script_data = ScriptData()
script_data.fetch_intraday_data("GOOGL")
script_data.convert_intraday_data("GOOGL")
script_data["GOOGL"]

,timestamp,open,high,low,close,volume
0,2023-02-06 17:00:00,102.90,103.170,102.5000,102.7300,684025
1,2023-02-06 18:00:00,102.76,102.900,102.7302,102.8055,13778
2,2023-02-06 19:00:00,102.89,103.340,102.8200,103.3400,24482
3,2023-02-06 20:00:00,103.33,103.340,103.0000,103.1600,18062
4,2023-02-07 05:00:00,103.01,104.000,103.0100,103.7000,44939
...,...,...,...,...,...,...
95,2023-02-14 16:00:00,94.16,94.845,93.7900,94.6300,9727151
96,2023-02-14 17:00:00,94.69,94.690,94.3200,94.3799,1831662
97,2023-02-14 18:00:00,94.38,94.400,94.3257,94.4000,71012
98,2023-02-14 19:00:00,94.36,94.490,94.2600,94.3600,39048


In [5]:
script_data.fetch_intraday_data("AAPL")
script_data.convert_intraday_data("AAPL")
script_data["AAPL"]

,timestamp,open,high,low,close,volume
0,2023-02-06 17:00:00,151.4993,151.7988,151.3495,151.4194,4683167
1,2023-02-06 18:00:00,151.4393,151.4993,151.3994,151.4693,28128
2,2023-02-06 19:00:00,151.4493,151.6590,151.4094,151.5092,45780
3,2023-02-06 20:00:00,151.5891,151.7688,151.5891,151.7489,48037
4,2023-02-07 05:00:00,151.5592,151.8587,151.4393,151.7289,47532
...,...,...,...,...,...,...
95,2023-02-14 16:00:00,153.2450,153.7300,152.9499,153.1000,7334349
96,2023-02-14 17:00:00,153.2000,153.2000,152.8500,153.0900,2612717
97,2023-02-14 18:00:00,153.0800,153.2000,153.0166,153.1800,259770
98,2023-02-14 19:00:00,153.2000,153.2000,153.0600,153.0900,40175


In [6]:
"GOOGL" in script_data

True

In [7]:
"AAPL" in script_data

True

In [8]:
"NVDA" in script_data

False

In [9]:
def indicator1(df: pd.DataFrame, timeperiod: int):
    data = {"timestamp": [], "indicator": []}
    moving_sum = 0

    # Create a moving sum that always has the sum of the timeperiod days ending at the current day
    
    for index in df.index:
        data["timestamp"].append(df["timestamp"][index])
        moving_sum += df["close"][index]
        if index+1 < timeperiod:
            data["indicator"].append(None)
        else:
            if index+1 > timeperiod:
                moving_sum -= df["close"][index-timeperiod]
            data["indicator"].append(moving_sum/timeperiod)

    return pd.DataFrame.from_dict(data)

In [10]:
indicator1(script_data["GOOGL"], 5)

,timestamp,indicator
0,2023-02-06 17:00:00,NaN
1,2023-02-06 18:00:00,NaN
2,2023-02-06 19:00:00,NaN
3,2023-02-06 20:00:00,NaN
4,2023-02-07 05:00:00,103.14710
...,...,...
95,2023-02-14 16:00:00,93.66562
96,2023-02-14 17:00:00,94.00946
97,2023-02-14 18:00:00,94.24696
98,2023-02-14 19:00:00,94.38596


In [11]:
indicator1(script_data["AAPL"], 5)

,timestamp,indicator
0,2023-02-06 17:00:00,NaN
1,2023-02-06 18:00:00,NaN
2,2023-02-06 19:00:00,NaN
3,2023-02-06 20:00:00,NaN
4,2023-02-07 05:00:00,151.57514
...,...,...
95,2023-02-14 16:00:00,152.63700
96,2023-02-14 17:00:00,152.91500
97,2023-02-14 18:00:00,153.13000
98,2023-02-14 19:00:00,153.14000


In [25]:
class Strategy:
    def __init__(self, script):
        self.script = script
        self.script_data = ScriptData()

    def get_script_data(self):
        self.script_data.fetch_intraday_data(self.script)
        self.script_data.convert_intraday_data(self.script)

    def get_signals(self):
        indicator_data = indicator1(self.script_data[self.script], 5)
        df = self.script_data[self.script]
        signals = pd.DataFrame(columns=["timestamp", "signal"])

        for index in indicator_data.index[1:]:
            if indicator_data["indicator"][index] == "nan":
                continue

            result = {
                "timestamp": [indicator_data["timestamp"][index]],
                "signal": ["NO_SIGNAL"]
            }

            # For BUY signal, previous day's close must be higher than indicator
            # and current day's close must be lower than indicator

            # For SELL signal, previous day's close must be lower than indicator
            # and current day's close must be greater than indicator
            if indicator_data["indicator"][index - 1] < df["close"][index-1] \
                    and df["close"][index] < indicator_data["indicator"][index]:
                result["signal"][0] = "BUY"
            elif indicator_data["indicator"][index - 1] > df["close"][index-1] \
                    and df["close"][index] > indicator_data["indicator"][index]:
                result["signal"][0] = "SELL"

            if result["signal"][0] != "NO_SIGNAL":
                signals = pd.concat([signals, pd.DataFrame.from_dict(result)], ignore_index=True)
        return signals

In [26]:
strategy = Strategy("NVDA")
strategy.get_script_data()
strategy.get_signals()

,timestamp,signal
0,2023-02-07 09:00:00,BUY
1,2023-02-07 11:00:00,SELL
2,2023-02-07 14:00:00,BUY
3,2023-02-07 15:00:00,SELL
4,2023-02-08 09:00:00,BUY
5,2023-02-08 10:00:00,SELL
6,2023-02-08 11:00:00,BUY
7,2023-02-08 13:00:00,SELL
8,2023-02-08 14:00:00,BUY
9,2023-02-08 20:00:00,SELL
